In [1]:
import requests
import logging
import pandas as pd
import time
from truedata.analytics import TD_analytics as TD
from datetime import datetime as dt,timedelta
from dateutil.relativedelta import relativedelta

# collect all the symbol

In [2]:
url = "https://api.truedata.in/getAllSymbols?segment=fo&user=True9001&password=praviin@9001&json=true&allexpiry=false&token=true&exchsymbol=NIFTY"
response = requests.get(url)
symbols = set(i[11] for i in response.json()['Records'])

In [3]:
from truedata.history import TD_hist
td_hist = TD_hist('True9001', 'praviin@9001', log_level=logging.WARNING)

(2024-07-25 10:24:50,782) WARNING :: Connected successfully to TrueData Historical Data Service...  (PID:15156 Thread:6216)


In [5]:
historical_data = {}
s = []

for symbol in symbols:
    data = td_hist.get_historic_data(symbol, start_time=dt(2024, 1, 1), end_time=dt(2024, 7, 15), bar_size="eod")

    if data is not None:
        historical_data[symbol] = data

        if data.get('timestamp') is not None:
            s.append(symbol)

(2024-07-25 10:26:10,432) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)
(2024-07-25 10:26:11,001) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)
(2024-07-25 10:26:16,966) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)
(2024-07-25 10:26:22,603) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)
(2024-07-25 10:26:31,213) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)
(2024-07-25 10:26:31,345) ERROR :: <class 'AttributeError'> -> 'DataFrame' object has no attribute 'timestamp' (PID:15156 Thread:6216)


# collect the hist. data which is not null

In [6]:
hist = {}
for i in s:
    
        
    data= td_hist.get_historic_data(i, start_time=dt.now() - relativedelta(months=7), bar_size='eod')
    if data is not None:
        hist[i] = data

# Collect the data which has 5% loss and its percentage at 5% and data after 15 days

In [7]:
import pandas as pd

# Initialize an empty list to store the results
results = []

for symbol in hist.keys():
    n = len(hist[symbol])

    for i in range(1, n):
        per_5_days = ((hist[symbol].loc[i]['close'] - hist[symbol].loc[i-1]['close']) / hist[symbol].loc[i-1]['close']) * 100

        if per_5_days <= -5:
            result = {
                'symbol': symbol,
                'location': i,
                'DATE': hist[symbol].loc[i]['timestamp'],
                'initial_close_amt': hist[symbol].loc[i-1]['close'],
                'final_close_amt': hist[symbol].loc[i]['close'],
                'per_5_days': per_5_days
            }
            
            # Check if we have enough data points for 15, 30, and 60 days calculations
            if i + 15 < n:
                per_15_days = ((hist[symbol].loc[i+15]['close'] - hist[symbol].loc[i]['close']) / hist[symbol].loc[i]['close']) * 100
                res_15 = {
                    'symbol': symbol,
                
                    
                    'initial_close_amt_at_5%_loss': hist[symbol].loc[i]['close'],
                    'final_close_amt_after_15_days_from_5_loss_price': hist[symbol].loc[i+15]['close'],
                    'percentage after 15 days': per_15_days
                }
                result.update(res_15)
                
            if i + 30 < n:
                per_30_days = ((hist[symbol].loc[i+30]['close'] - hist[symbol].loc[i]['close']) / hist[symbol].loc[i]['close']) * 100
                res_30 = {
                    'symbol': symbol,
        
                    
                    'initial_close_amt_at_5%_loss': hist[symbol].loc[i]['close'],
                    'final_close_amt_after_30_days_from_5_loss_price': hist[symbol].loc[i+30]['close'],
                    'percentage after 30 days': per_30_days
                }
                result.update(res_30)
                
            if i + 60 < n:
                per_60_days = ((hist[symbol].loc[i+60]['close'] - hist[symbol].loc[i]['close']) / hist[symbol].loc[i]['close']) * 100
                res_60 = {
                    'symbol': symbol,
                    
            
                    'initial_close_amt_at_5%_loss': hist[symbol].loc[i]['close'],
                    'final_close_amt_after_60_days_from_5_loss_price': hist[symbol].loc[i+60]['close'],
                    'percentage after 60 days': per_60_days
                }
                result.update(res_60)
                
            results.append(result)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)


(results_df)

,symbol,location,DATE,initial_close_amt,final_close_amt,per_5_days,initial_close_amt_at_5%_loss,final_close_amt_after_15_days_from_5_loss_price,percentage after 15 days,final_close_amt_after_30_days_from_5_loss_price,percentage after 30 days,final_close_amt_after_60_days_from_5_loss_price,percentage after 60 days
0,INDUSTOWER,33,2024-02-12,226.15,213.55,-5.571523,213.55,265.70,24.420510,283.65,32.826036,328.3,53.734488
1,INDUSTOWER,109,2024-06-04,364.00,307.95,-15.398352,307.95,356.10,15.635655,418.95,36.044812,NaN,NaN
2,INDUSINDBK,20,2024-01-23,1535.35,1441.70,-6.099586,1441.70,1481.50,2.760630,1542.45,6.988278,1474.5,2.275092
3,INDUSINDBK,109,2024-06-04,1529.70,1391.50,-9.034451,1391.50,1497.90,7.646425,1456.90,4.699964,NaN,NaN
4,MGL,51,2024-03-06,1565.40,1329.60,-15.063243,1329.60,1385.00,4.166667,1422.35,6.975782,1360.2,2.301444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,HAVELLS,109,2024-06-04,1879.30,1773.00,-5.656361,1773.00,1918.30,8.195149,1862.65,5.056402,NaN,NaN
303,HAVELLS,140,2024-07-19,1862.65,1768.50,-5.054626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,CUB,99,2024-05-21,153.10,145.00,-5.290660,145.00,149.47,3.082759,170.07,17.289655,NaN,NaN
305,CUB,109,2024-06-04,147.20,135.55,-7.914402,135.55,167.55,23.607525,158.86,17.196606,NaN,NaN


# Sort the above data by DATE

In [8]:
df=results_df.sort_values(by='DATE')
df

,symbol,location,DATE,initial_close_amt,final_close_amt,per_5_days,initial_close_amt_at_5%_loss,final_close_amt_after_15_days_from_5_loss_price,percentage after 15 days,final_close_amt_after_30_days_from_5_loss_price,percentage after 30 days,final_close_amt_after_60_days_from_5_loss_price,percentage after 60 days
255,IDEA,5,2024-01-02,17.00,16.05,-5.588235,16.05,14.40,-10.280374,15.55,-3.115265,13.25,-17.445483
150,NATIONALUM,6,2024-01-03,139.05,129.85,-6.616325,129.85,135.60,4.428186,157.25,21.101271,161.50,24.374278
247,NAVINFLUOR,9,2024-01-08,3822.65,3609.70,-5.570743,3609.70,3308.05,-8.356650,3203.50,-11.253013,3132.95,-13.207469
240,HINDCOPPER,9,2024-01-08,283.30,267.85,-5.453583,267.85,285.60,6.626843,250.10,-6.626843,313.95,17.211126
284,BANDHANBNK,9,2024-01-08,251.70,233.10,-7.389750,233.10,223.45,-4.139854,202.70,-13.041613,197.95,-15.079365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,DIXON,140,2024-07-19,11945.85,11267.20,-5.681052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,CUMMINSIND,140,2024-07-19,3796.85,3559.45,-6.252551,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,WIPRO,141,2024-07-22,557.20,505.80,-9.224695,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,MANAPPURAM,142,2024-07-23,215.27,202.73,-5.825243,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df.sort_values(by='DATE').merge(df.groupby('DATE').size().reset_index(name='No. of stock fall by 5%'), on='DATE', how='left')

# Unique Row group by using the Date (we get Total number of stock on this date)

In [16]:
a=df.groupby('DATE').size().reset_index(name='No. of stock fall by 5%')
a


,DATE,No. of stock fall by 5%
0,2024-01-02,1
1,2024-01-03,1
2,2024-01-08,3
3,2024-01-09,2
4,2024-01-11,1
...,...,...
57,2024-07-18,2
58,2024-07-19,10
59,2024-07-22,1
60,2024-07-23,1


In [17]:
df1=df[['DATE', 'No. of stock fall by 5%']].drop_duplicates()

In [18]:
# Reset index
df1.reset_index(drop=True, inplace=True)

In [19]:
df1

,DATE,No. of stock fall by 5%
0,2024-01-02,1
1,2024-01-03,1
2,2024-01-08,3
3,2024-01-09,2
4,2024-01-11,1
...,...,...
57,2024-07-18,2
58,2024-07-19,10
59,2024-07-22,1
60,2024-07-23,1


In [21]:
# Initialize the cum_sum_after_15_days column with zeros
df1['cum_sum_after_15_days'] = 0.0
# Calculate the cumulative sum for the first 15 days
df1.iloc[:16, df1.columns.get_loc('cum_sum_after_15_days')] = df1['No. of stock fall by 5%'].iloc[:16].cumsum()


In [22]:
df1.head(20)

,DATE,No. of stock fall by 5%,cum_sum_after_15_days
0,2024-01-02,1,1.0
1,2024-01-03,1,2.0
2,2024-01-08,3,5.0
3,2024-01-09,2,7.0
4,2024-01-11,1,8.0
5,2024-01-12,1,9.0
6,2024-01-17,3,12.0
7,2024-01-18,3,15.0
8,2024-01-23,28,43.0
9,2024-01-25,4,47.0


In [29]:

# Adjust cumulative sum for days beyond the 15th day
for i in range(16, len(df1)):
    df1.loc[i, 'cum_sum_after_15_days'] = df1.loc[i-1, 'cum_sum_after_15_days'] + df1.loc[i, 'No. of stock fall by 5%'] - df1.loc[i-16, 'No. of stock fall by 5%']
df1

,DATE,No. of stock fall by 5%,cum_sum_after_15_days
0,2024-01-02,1,1.0
1,2024-01-03,1,2.0
2,2024-01-08,3,5.0
3,2024-01-09,2,7.0
4,2024-01-11,1,8.0
...,...,...,...
57,2024-07-18,2,136.0
58,2024-07-19,10,140.0
59,2024-07-22,1,138.0
60,2024-07-23,1,138.0


In [30]:
df1.head(20)

,DATE,No. of stock fall by 5%,cum_sum_after_15_days
0,2024-01-02,1,1.0
1,2024-01-03,1,2.0
2,2024-01-08,3,5.0
3,2024-01-09,2,7.0
4,2024-01-11,1,8.0
5,2024-01-12,1,9.0
6,2024-01-17,3,12.0
7,2024-01-18,3,15.0
8,2024-01-23,28,43.0
9,2024-01-25,4,47.0


In [31]:
df2=df[['symbol','DATE','final_close_amt']]
df2

,symbol,DATE,final_close_amt
0,IDEA,2024-01-02,16.05
1,NATIONALUM,2024-01-03,129.85
2,NAVINFLUOR,2024-01-08,3609.70
3,HINDCOPPER,2024-01-08,267.85
4,BANDHANBNK,2024-01-08,233.10
...,...,...,...
302,DIXON,2024-07-19,11267.20
303,CUMMINSIND,2024-07-19,3559.45
304,WIPRO,2024-07-22,505.80
305,MANAPPURAM,2024-07-23,202.73


In [32]:
df2.head(20)

,symbol,DATE,final_close_amt
0,IDEA,2024-01-02,16.05
1,NATIONALUM,2024-01-03,129.85
2,NAVINFLUOR,2024-01-08,3609.70
3,HINDCOPPER,2024-01-08,267.85
4,BANDHANBNK,2024-01-08,233.10
5,IDEA,2024-01-09,16.20
6,POLYCAB,2024-01-09,4867.10
7,POLYCAB,2024-01-11,3878.45
8,METROPOLIS,2024-01-12,1503.80
9,SAIL,2024-01-17,113.10


# store all the symbol inside one variable

In [33]:
# Store all symbols in one variable
df_symbols = df2['symbol'].tolist()

In [34]:
# Sort each DataFrame by the 'timestamp' column
for symbol, df in hist.items():
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Convert 'timestamp' to datetime
    df.sort_values(by='timestamp', inplace=True)      # Sort by 'timestamp'
    hist[symbol] = df 
    


In [35]:
idea=hist['IDEA']
idea

,timestamp,open,high,low,close,volume,oi
0,2023-12-26,13.65,13.75,13.30,13.35,328954630,0
1,2023-12-27,13.40,13.50,13.20,13.45,139443887,0
2,2023-12-28,13.50,13.60,13.10,13.25,169009935,0
3,2023-12-29,13.45,16.25,13.30,16.00,1688819452,0
4,2024-01-01,16.20,18.40,15.55,17.00,1784392277,0
...,...,...,...,...,...,...,...
140,2024-07-19,16.37,16.43,15.82,15.87,368882386,0
141,2024-07-22,15.77,16.05,15.65,15.89,529018766,0
142,2024-07-23,15.90,15.91,14.57,15.28,1127413061,0
143,2024-07-24,15.30,15.66,15.21,15.58,453449137,0


# Convert timestamps to datetime and sort them

In [36]:
# Convert timestamps to datetime and sort them
for symbol in hist:
    hist[symbol]['timestamp'] = pd.to_datetime(hist[symbol]['timestamp'])
    hist[symbol].sort_values(by='timestamp', inplace=True)

# Function to get the date after 15 days from the hist data

In [43]:

def get_date_after_15_days(start_date):
    dates = idea['timestamp']
    start_date = pd.to_datetime(start_date)
    filtered_dates = dates[dates > start_date]
    if len(filtered_dates) >= 15:
        date_after_15_days = filtered_dates.iloc[14]  # 15th day after start_date
        return date_after_15_days
    else:
        return None

# Add new column with date after 15 days
df2['date_after_15_days'] = df2['DATE'].apply(get_date_after_15_days)

C:\Users\ABC\AppData\Local\Temp\ipykernel_15156\3800658744.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date_after_15_days'] = df2['DATE'].apply(get_date_after_15_days)


In [45]:
# Convert timestamps to datetime and sort them for each symbol in hist
for symbol in hist:
    hist[symbol]['timestamp'] = pd.to_datetime(hist[symbol]['timestamp'])
    hist[symbol].sort_values(by='timestamp', inplace=True)

# Function to get the closing price from hist data for a specific symbol and date
def get_closing_price(symbol, date):
    date = pd.to_datetime(date)
    closing_price = hist[symbol][hist[symbol]['timestamp'] == date]['close']
    return closing_price.iloc[0] if not closing_price.empty else None

# Apply the function to df2 to get the closing prices after 15 days
df2['after_15_days_price'] = df2.apply(lambda row: get_closing_price(row['symbol'], row['date_after_15_days']), axis=1)

C:\Users\ABC\AppData\Local\Temp\ipykernel_15156\3607012688.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['after_15_days_price'] = df2.apply(lambda row: get_closing_price(row['symbol'], row['date_after_15_days']), axis=1)


In [47]:
df2

,symbol,DATE,final_close_amt,date_after_15_days,after_15_days_price
0,IDEA,2024-01-02,16.05,2024-01-23,14.40
1,NATIONALUM,2024-01-03,129.85,2024-01-24,135.60
2,NAVINFLUOR,2024-01-08,3609.70,2024-01-30,3308.05
3,HINDCOPPER,2024-01-08,267.85,2024-01-30,285.60
4,BANDHANBNK,2024-01-08,233.10,2024-01-30,223.45
...,...,...,...,...,...
302,DIXON,2024-07-19,11267.20,NaT,NaN
303,CUMMINSIND,2024-07-19,3559.45,NaT,NaN
304,WIPRO,2024-07-22,505.80,NaT,NaN
305,MANAPPURAM,2024-07-23,202.73,NaT,NaN


In [48]:
df1

,DATE,No. of stock fall by 5%,cum_sum_after_15_days
0,2024-01-02,1,1.0
1,2024-01-03,1,2.0
2,2024-01-08,3,5.0
3,2024-01-09,2,7.0
4,2024-01-11,1,8.0
...,...,...,...
57,2024-07-18,2,136.0
58,2024-07-19,10,140.0
59,2024-07-22,1,138.0
60,2024-07-23,1,138.0


In [49]:
# Merge df2 with df1 on date_after_15_days and DATE to add cum_sum_after_15_days column
df2 = df2.merge(df1[['DATE', 'cum_sum_after_15_days']], left_on='date_after_15_days', right_on='DATE', suffixes=('', '_after_15_days'))

In [51]:
df2.to_csv("25th_july_senario.csv")

In [76]:
# Merge df2 with df1 on DATE column to add cum_sum_after_15_days column
df2 = df2.merge(df1[['DATE', 'No. of stock fall by 5%','cum_sum_after_15_days']], on='DATE', how='left')

In [89]:
df2.head(60)

,symbol,DATE,final_close_amt,date_after_15_days,after_15_days_price,cum_sum_after_15_days_x,No. of stock fall by 5%
0,IDEA,2024-01-02,16.05,2024-01-23,14.40,43.0,1
1,HINDCOPPER,2024-01-08,267.85,2024-01-30,285.60,52.0,3
2,BANDHANBNK,2024-01-08,233.10,2024-01-30,223.45,52.0,3
3,NAVINFLUOR,2024-01-08,3609.70,2024-01-30,3308.05,52.0,3
4,METROPOLIS,2024-01-12,1503.80,2024-02-05,1666.80,56.0,1
5,IEX,2024-01-17,147.25,2024-02-08,144.45,59.0,3
6,HDFCBANK,2024-01-17,1537.50,2024-02-08,1403.05,59.0,3
7,SAIL,2024-01-17,113.10,2024-02-08,139.05,59.0,3
8,LTIM,2024-01-18,5603.00,2024-02-09,5453.30,64.0,3
9,ICICIPRULI,2024-01-18,486.20,2024-02-09,521.45,64.0,3


In [86]:
df2=df2.drop(columns='DATE_after_15_days')



In [79]:
df1.head(30)

,DATE,No. of stock fall by 5%,cum_sum_after_15_days
0,2024-01-02,1,1.0
1,2024-01-03,1,2.0
2,2024-01-08,3,5.0
3,2024-01-09,2,7.0
4,2024-01-11,1,8.0
5,2024-01-12,1,9.0
6,2024-01-17,3,12.0
7,2024-01-18,3,15.0
8,2024-01-23,28,43.0
9,2024-01-25,4,47.0


In [67]:
hist['NATIONALUM'].loc[5:30]

,timestamp,open,high,low,close,volume,oi
5,2024-01-02,136.60,139.85,131.80,139.05,51857420,0
6,2024-01-03,138.50,138.50,129.30,129.85,59450714,0
7,2024-01-04,131.00,132.90,129.65,130.50,22117963,0
8,2024-01-05,131.45,132.80,127.20,128.10,21198368,0
9,2024-01-08,128.95,129.50,123.05,123.75,18668735,0
10,2024-01-09,125.00,129.70,123.05,128.45,27422652,0
11,2024-01-10,130.70,133.25,127.00,130.70,44056844,0
12,2024-01-11,131.10,131.70,128.80,130.40,13675789,0
13,2024-01-12,131.90,133.45,130.80,131.45,17384204,0
14,2024-01-15,132.60,135.70,129.30,135.30,31549703,0


In [90]:
df2=results_df

In [91]:
df2[['symbol','DATE','final_close_amt']].sort_values(by='DATE').tail(60)

,symbol,DATE,final_close_amt
26,UPL,2024-06-04,495.95
27,INDIGO,2024-06-04,4073.20
29,IDFC,2024-06-04,107.25
30,SBIN,2024-06-04,775.20
33,OBEROIRLTY,2024-06-04,1745.25
34,HAVELLS,2024-06-04,1773.00
37,BALRAMCHIN,2024-06-04,364.10
39,LT,2024-06-04,3403.20
40,SIEMENS,2024-06-04,6372.15
46,CUB,2024-06-04,135.55
